''' -- coding:utf-8 -- @author: Huaiyu Dong ＆ Yixing Huang @Time: 2023-10-23 @Explanition: This code aims to predict the EAB of microwave absorber structure using the DNN neural network architecture with GA algorithm. '''

In [ ]:
###归一化、无数据增强功能###
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

# Set random seed for PyTorch and NumPy
#seed = 14618
#torch.manual_seed(seed)
#np.random.seed(seed)

# Load your data from mydata_petals.csv
data = pd.read_csv('mydata_petals.csv')

# Extract the last column (EAB) as the target variable
EAB = data.iloc[:, -1].values

# Extract the first 10 columns as input features
input = data.iloc[:, :-1].values

# Create MinMaxScaler objects for input and EAB
scaler_in = MinMaxScaler(feature_range=(-0.5, 0.5))
scaler_out = MinMaxScaler(feature_range=(-0.99, 0.99))

# Fit and transform input data, and store the transformation matrix ps_in
p_scaled = scaler_in.fit_transform(input)
p = torch.FloatTensor(p_scaled.T)
ps_in = scaler_in.scale_

# Fit and transform EAB data, and store the transformation matrix ts_out
t_scaled = scaler_out.fit_transform(EAB.reshape(-1, 1))
t = torch.FloatTensor(t_scaled)
ts_out = scaler_out.scale_

print("Scaled Data Shapes:")
print(p_scaled.shape)
print(t_scaled.shape)

# Define the ratio for training and testing data
train_ratio = 0.8  # 80% training data, 
test_ratio = 0.2   # 20% testing data

# Number of samples
n_samples = len(EAB)

# Create a random permutation of indices for shuffling
indices = np.random.permutation(n_samples)

# Calculate the sizes of the training and test sets
n_train = int(train_ratio * n_samples)
n_test = n_samples - n_train

# Use the shuffled indices to split the data into training and test sets
train_indices = indices[:n_train]
test_indices = indices[n_train:]

# Split the data into training and test sets based on the shuffled indices
X_train, y_train = p_scaled[train_indices], t_scaled[train_indices]
X_test, y_test = p_scaled[test_indices], t_scaled[test_indices]

print("Training Data Shapes:")
print(X_train.shape, y_train.shape)
print("Test Data Shapes:")
print(X_test.shape, y_test.shape)

# 打印训练数据前五行
print("First 5 rows of Training Data:")
print(X_train[:5])
print("First 5 rows of Training Target (y_train):")
print(y_train[:5])

print("Test Data Shapes:")
print(X_test.shape, y_test.shape)

# 打印测试数据前五行
print("First 5 rows of Test Data:")
print(X_test[:5])
print("First 5 rows of Test Target (y_test):")
print(y_test[:5])

In [ ]:
#CPU-version-仅使用PSO完成DNN权重和偏差的初始化更新##
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time

# Define your DNN architecture
class DNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super(DNN, self).__init__()
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList()
        for i in range(1, len(dims)):
            self.layers.append(nn.Linear(dims[i - 1], dims[i]))
            if i < len(dims) - 1:
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Define the fitness function for PSO
def fitness_function(weights, dnn, criterion, inputs, targets):
    # Apply weights to the DNN model
    start_idx = 0
    for param in dnn.parameters():
        param_size = param.numel()
        param.data = torch.FloatTensor(weights[start_idx:start_idx+param_size]).view(param.size())
        start_idx += param_size

    # Forward pass
    outputs = dnn(inputs)

    # Calculate the loss
    loss = criterion(outputs, targets)

    return loss.item()

# Define input dimension and hidden layer dimensions (adjust as needed)
input_dim = 10  
hidden_dims = [14, 12, 8, 3]  
output_dim = 1  

# Add the following lines to define 'X_train' and 'y_train' based on your data
X_train = X_train  
y_train = y_train 

# Initialize the DNN
dnn = DNN(input_dim, output_dim, hidden_dims)
criterion = nn.MSELoss()

# Define the problem bounds for PSO
num_weights = sum(p.numel() for p in dnn.parameters())
lb = [-1.0] * num_weights  # Lower bounds for weights
ub = [1.0] * num_weights  # Upper bounds for weights

# Initialize PSO parameters
swarm_size = 30
max_iters = 10
c1 = 1.5  # Cognitive coefficient
c2 = 1.5 # Social coefficient
w = 0.3   # Inertia weight

# Record the start time
start_time = time.time()

# Initialize the swarm  
swarm = np.random.uniform(lb, ub, (swarm_size, num_weights))
velocities = np.zeros((swarm_size, num_weights))
pbest_positions = swarm.copy()
pbest_scores = np.zeros(swarm_size)
gbest_position = None
gbest_score = float('inf')


for i in range(max_iters):
    for j in range(swarm_size):  #最小化损失函数
        score = fitness_function(swarm[j], dnn, criterion, torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        if score < pbest_scores[j]:
            pbest_scores[j] = score
            pbest_positions[j] = swarm[j]
        
        if score < gbest_score:   #最小化损失函数
            gbest_score = score
            gbest_position = swarm[j]
        
    for j in range(swarm_size):
        r1 = np.random.rand(num_weights)
        r2 = np.random.rand(num_weights)
        velocities[j] = w * velocities[j] + c1 * r1 * (pbest_positions[j] - swarm[j]) + c2 * r2 * (gbest_position - swarm[j])
        swarm[j] += velocities[j]

# Update the DNN with the global best position
start_idx = 0
for param in dnn.parameters():
    param_size = param.numel()
    param.data = torch.FloatTensor(gbest_position[start_idx:start_idx+param_size]).view(param.size())
    start_idx += param_size

# Training with optimized weights
num_epochs = 300

optimizer = optim.Adam(dnn.parameters(), lr=0.02)

losses = []

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = dnn(torch.FloatTensor(X_train))
    loss = criterion(outputs, torch.FloatTensor(y_train))
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
# Record the end time
end_time = time.time()
# Calculate and print the training duration
training_duration = end_time - start_time
print(f"Training duration: {training_duration:.2f} seconds")

# Plot the loss curve
plt.figure()
plt.plot(range(1, num_epochs+1), losses, marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()
print(dnn)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
with torch.no_grad():
    predicted_train = dnn(torch.FloatTensor(X_train))
    train_true_values = y_train
    r_train = r2_score(train_true_values, predicted_train)
    mae_train = mean_absolute_error(train_true_values, predicted_train)   # MAE training set
    mse_train = mean_squared_error(train_true_values, predicted_train)    # MSE training set
    rmse_train = math.sqrt(mse_train)                               # RMSE training set

print(f"R-squared (r_train) on training set: {r_train:.3f}")
print(f"Mean Absolute Error (MAE) on training set: {mae_train:.3f}")
print(f"Mean Squared Error (MSE) on training set: {mse_train:.3f}")
print(f"Root Mean Squared Error (RMSE) on training set: {rmse_train:.3f}")

with torch.no_grad():              # Perform regression analysis on the test set (X_test, y_test)
    predicted_test = dnn(torch.FloatTensor(X_test))
    test_true_values = y_test
    r_test = r2_score(test_true_values, predicted_test)
    mae_test = mean_absolute_error(test_true_values, predicted_test)   #MAE test set
    mse_test = mean_squared_error(test_true_values, predicted_test)    # MSE test set
    rmse_test = math.sqrt(mse_test)                               #RMSE test set
    predicted_train = dnn(torch.FloatTensor(X_train))

print(f"R-squared (r_test) on test set: {r_test:.3f}")
print(f"Mean Absolute Error (MAE) on test set: {mae_test:.3f}")
print(f"Mean Squared Error (MSE) on test set: {mse_test:.3f}")
print(f"Root Mean Squared Error (RMSE) on test set: {rmse_test:.3f}")

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

actual_train = y_train
actual_test = y_test  # Replace 'y_test' with your test labels

# Create a figure with subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Scatter plot of actual values and regression line for training set
axes[0].scatter(actual_train, actual_train, color='blue', label='Actual', marker='o', s=10)
axes[0].scatter(actual_train, predicted_train, color='red', label='Predicted', marker='x', s=10)
axes[0].set_xlim(-1.2, 1.2)
axes[0].set_ylim(-1.2, 1.2)
correlation_train = np.corrcoef(actual_train, predicted_train, rowvar=False)[0, 1]
axes[0].set_title(f'Training Set\nCorrelation: {correlation_train:.3f}')
regression_train = LinearRegression()
regression_train.fit(actual_train.reshape(-1, 1), predicted_train)
regression_line_train = regression_train.predict(actual_train.reshape(-1, 1))
axes[0].plot(actual_train, regression_line_train, color='green', linestyle='--', label='Regression Line')
axes[0].legend()

# Scatter plot of actual values and regression line for test set
axes[1].scatter(actual_test, actual_test, color='blue', label='Actual', marker='o', s=10)
axes[1].scatter(actual_test, predicted_test, color='red', label='Predicted', marker='x', s=10)
axes[1].set_xlim(-1.2, 1.2)
axes[1].set_ylim(-1.2, 1.2)
correlation_test = np.corrcoef(actual_test, predicted_test, rowvar=False)[0, 1]
axes[1].set_title(f'Test Set\nCorrelation: {correlation_test:.3f}')
regression_test = LinearRegression()
regression_test.fit(actual_test.reshape(-1, 1), predicted_test)
regression_line_test = regression_test.predict(actual_test.reshape(-1, 1))
axes[1].plot(actual_test, regression_line_test, color='green', linestyle='--', label='Regression Line')
axes[1].legend()
plt.tight_layout()
plt.show()

#GPU-version-仅使用PSO完成DNN权重和偏差的初始化更新##
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


# Define your DNN architecture
class DNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super(DNN, self).__init__()
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList()
        for i in range(1, len(dims)):
            self.layers.append(nn.Linear(dims[i - 1], dims[i]))
            if i < len(dims) - 1:
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Define the fitness function for PSO
def fitness_function(weights, dnn, criterion, inputs, targets):
    # Apply weights to the DNN model
    start_idx = 0
    for param in dnn.parameters():
        param_size = param.numel()
        param.data = torch.FloatTensor(weights[start_idx:start_idx+param_size]).view(param.size())
        param.data = param.data.to('cuda')  # Move data to GPU
        start_idx += param_size

    # Forward pass
    outputs = dnn(inputs)

    # Calculate the loss
    loss = criterion(outputs, targets)

    return loss.item()

# Define input dimension and hidden layer dimensions (adjust as needed)
input_dim = 10  # Adjust based on your data
hidden_dims = [12, 10, 6, 3]  # Adjust based on your desired architecture
output_dim = 1  # Adjust based on your data

# Add the following lines to define 'X_train' and 'y_train' based on your data
X_train = X_train  # Replace with your input data
y_train = y_train  # Replace with your output data

# Initialize the DNN
dnn = DNN(input_dim, output_dim, hidden_dims).to('cuda')
criterion = nn.MSELoss()

# Define the problem bounds for PSO
num_weights = sum(p.numel() for p in dnn.parameters())
lb = [-1.0] * num_weights  # Lower bounds for weights
ub = [1.0] * num_weights  # Upper bounds for weights

# Initialize PSO parameters
swarm_size = 100
max_iters = 100
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient
w = 0.7    # Inertia weight

# Initialize the swarm
swarm = np.random.uniform(lb, ub, (swarm_size, num_weights))
velocities = np.zeros((swarm_size, num_weights))
pbest_positions = swarm.copy()
pbest_scores = np.zeros(swarm_size)
gbest_position = None
gbest_score = float('inf')

for i in range(max_iters):
    for j in range(swarm_size):
        score = fitness_function(swarm[j], dnn, criterion, torch.FloatTensor(X_train).to('cuda'), torch.FloatTensor(y_train).to('cuda'))
        if score < pbest_scores[j]:
            pbest_scores[j] = score
            pbest_positions[j] = swarm[j]
        
        if score < gbest_score:
            gbest_score = score
            gbest_position = swarm[j]
        
    for j in range(swarm_size):
        r1 = np.random.rand(num_weights)
        r2 = np.random.rand(num_weights)
        velocities[j] = w * velocities[j] + c1 * r1 * (pbest_positions[j] - swarm[j]) + c2 * r2 * (gbest_position - swarm[j])
        swarm[j] += velocities[j]

# Update the DNN with the global best position
start_idx = 0
for param in dnn.parameters():
    param_size = param.numel()
    param.data = torch.FloatTensor(gbest_position[start_idx:start_idx+param_size]).view(param.size())
    start_idx += param_size

# Training with optimized weights
optimizer = optim.Adam(dnn.parameters(), lr=0.01)

losses = []

for epoch in range(3000):
    optimizer.zero_grad()
    outputs = dnn(torch.FloatTensor(X_train).to('cuda'))
    loss = criterion(outputs, torch.FloatTensor(y_train).to('cuda'))
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

# Plot the loss curve
plt.figure()
plt.plot(range(1, 3001), losses, marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()

#使用PSO算法完成所有训练轮次权重和偏差的更新#
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Define your DNN architecture
class DNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims):
        super(DNN, self).__init__()
        dims = [input_dim] + hidden_dims + [output_dim]
        self.layers = nn.ModuleList()
        for i in range(1, len(dims)):
            self.layers.append(nn.Linear(dims[i - 1], dims[i]))
            if i < len(dims) - 1:
                self.layers.append(nn.Tanh())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Define the fitness function for PSO
def fitness_function(weights, dnn, criterion, inputs, targets):
    # Apply weights to the DNN model
    start_idx = 0
    for param in dnn.parameters():
        param_size = param.numel()
        param.data = torch.FloatTensor(weights[start_idx:start_idx+param_size]).view(param.size())
        start_idx += param_size

    # Forward pass
    outputs = dnn(inputs)

    # Calculate the loss
    loss = criterion(outputs, targets)

    return loss.item()

# Define input dimension and hidden layer dimensions (adjust as needed)
input_dim = 10  # Adjust based on your data
hidden_dims = [12, 10, 8, 6]  # Adjust based on your desired architecture
output_dim = 1  # Adjust based on your data

# Add the following lines to define 'X_train' and 'y_train' based on your data
X_train = X_train  # Replace with your input data
y_train = y_train  # Replace with your output data

# Initialize the DNN
dnn = DNN(input_dim, output_dim, hidden_dims)
criterion = nn.MSELoss()

# Define the problem bounds for PSO
num_weights = sum(p.numel() for p in dnn.parameters())
lb = [-1.0] * num_weights  # Lower bounds for weights
ub = [1.0] * num_weights  # Upper bounds for weights

# Initialize PSO parameters
swarm_size = 50
max_iters = 100
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient
w = 0.7    # Inertia weight

# Training with optimized weights
optimizer = optim.Adam(dnn.parameters(), lr=0.01)

losses = []

for i in range(max_iters):
    # PSO optimization
    swarm = np.random.uniform(lb, ub, (swarm_size, num_weights))
    velocities = np.zeros((swarm_size, num_weights))
    pbest_positions = swarm.copy()
    pbest_scores = np.zeros(swarm_size)
    gbest_position = None
    gbest_score = float('inf')

    for j in range(1000):  # Inner training loop
        for k in range(swarm_size):
            score = fitness_function(swarm[k], dnn, criterion, torch.FloatTensor(X_train), torch.FloatTensor(y_train))
            if score < pbest_scores[k]:
                pbest_scores[k] = score
                pbest_positions[k] = swarm[k]

            if score < gbest_score:
                gbest_score = score
                gbest_position = swarm[k]

        for k in range(swarm_size):
            r1 = np.random.rand(num_weights)
            r2 = np.random.rand(num_weights)
            velocities[k] = w * velocities[k] + c1 * r1 * (pbest_positions[k] - swarm[k]) + c2 * r2 * (gbest_position - swarm[k])
            swarm[k] += velocities[k]

        # Update the DNN with the global best position
        start_idx = 0
        for param in dnn.parameters():
            param_size = param.numel()
            param.data = torch.FloatTensor(gbest_position[start_idx:start_idx+param_size]).view(param.size())
            start_idx += param_size

        # Calculate and store loss
        optimizer.zero_grad()
        outputs = dnn(torch.FloatTensor(X_train))
        loss = criterion(outputs, torch.FloatTensor(y_train))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

# Plot the loss curve
plt.figure()
plt.plot(range(1, len(losses) + 1), losses, marker='o', linestyle='-')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()